In [7]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

import tensorflow as tf
from tensorflow import keras

from keras.preprocessing.image import load_img

from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input
from keras.applications.xception import decode_predictions

from keras import layers


from tqdm.auto import tqdm 
import time

In [8]:
## Prep

In [9]:
input_size = 299

In [17]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    './clothing-dataset-small/train',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(input_size, input_size),
    shuffle=True
)

data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomZoom(0.1),
])

def augment_and_preprocess(image, label):
    image = data_augmentation(image)
    image = preprocess_input(image)
    return image, label

train_ds = train_ds.map(augment_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.prefetch(buffer_size=tf.data.AUTOTUNE)

Found 3068 files belonging to 10 classes.


In [18]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    './clothing-dataset-small/validation',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(input_size, input_size),
    shuffle=False
)

data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomZoom(0.1),
])

def augment_and_preprocess(image, label):
    image = data_augmentation(image)
    image = preprocess_input(image)
    return image, label

val_ds = val_ds.map(augment_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=tf.data.AUTOTUNE)

Found 341 files belonging to 10 classes.


In [19]:
## Build Model

In [20]:
def make_model(input_size=150, learning_rate=0.01, size_inner=100, drop_rate=0.5):

    base_model = Xception(
        weights='imagenet', 
        include_top=False, 
        input_shape=(input_size, input_size, 3)
    )
    base_model.trainable = False
    
    inputs = keras.Input(shape=(input_size, input_size, 3))
    base = base_model(inputs, training=False) 
    vectors = keras.layers.GlobalAveragePooling2D()(base)

    inner = keras.layers.Dense(size_inner, activation='relu')(vectors)
    drop = keras.layers.Dropout(drop_rate)(inner)
    
    outputs = keras.layers.Dense(10)(drop)
    model = keras.Model(inputs, outputs)

    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    loss = keras.losses.CategoricalCrossentropy(from_logits=True)
    model.compile(
        optimizer=optimizer, 
        loss=loss, 
        metrics=['accuracy']
    )

    return model

In [21]:
checkpoint = keras.callbacks.ModelCheckpoint(
    'xception_v5_{epoch:02d}_{val_accuracy:.3f}.keras', 
    save_best_only=True,
    monitor='val_accuracy',
    mode='max'
)

In [22]:
drop_rate = 0.2
learning_rate = 0.0005
size=50

model = make_model(
    input_size=299,
    learning_rate=learning_rate,
    size_inner=size,
    drop_rate=drop_rate
)
history = model.fit(
    train_ds, 
    epochs=50, 
    validation_data=val_ds,
    callbacks=[checkpoint]
)

Epoch 1/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 120s 1s/step - accuracy: 0.4782 - loss: 1.6128 - val_accuracy: 0.7889 - val_loss: 0.6887
Epoch 2/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 119s 1s/step - accuracy: 0.7808 - loss: 0.6743 - val_accuracy: 0.8270 - val_loss: 0.5239
Epoch 3/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 119s 1s/step - accuracy: 0.8377 - loss: 0.5108 - val_accuracy: 0.8622 - val_loss: 0.4480
Epoch 4/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 117s 1s/step - accuracy: 0.8633 - loss: 0.4378 - val_accuracy: 0.8534 - val_loss: 0.4319
Epoch 5/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 117s 1s/step - accuracy: 0.8567 - loss: 0.4173 - val_accuracy: 0.8563 - val_loss: 0.4165
Epoch 6/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 117s 1s/step - accuracy: 0.8896 - loss: 0.3524 - val_accuracy: 0.8798 - val_loss: 0.3879
Epoch 7/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 118s 1s/step - accuracy: 0.8838 - loss: 0.3457 - val_accuracy: 0.8622 - val_loss: 0.3940
Epoch 8/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 114s 1s/step - accuracy: 0.8941 - loss: 0.3055 - val_accuracy: 0.8768 - v

KeyboardInterrupt: 

```
# Without Data Aug
Epoch 1/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 31s 314ms/step - accuracy: 0.5522 - loss: 1.3894 - val_accuracy: 0.7801 - val_loss: 0.6842
Epoch 2/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 30s 310ms/step - accuracy: 0.7555 - loss: 0.6901 - val_accuracy: 0.8123 - val_loss: 0.5758
Epoch 3/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 30s 311ms/step - accuracy: 0.8080 - loss: 0.5301 - val_accuracy: 0.8006 - val_loss: 0.5855
Epoch 4/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 30s 317ms/step - accuracy: 0.8429 - loss: 0.4260 - val_accuracy: 0.7977 - val_loss: 0.5529
Epoch 5/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 31s 324ms/step - accuracy: 0.8910 - loss: 0.3264 - val_accuracy: 0.8211 - val_loss: 0.5373
Epoch 6/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 30s 316ms/step - accuracy: 0.9096 - loss: 0.2716 - val_accuracy: 0.8211 - val_loss: 0.5518
Epoch 7/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 32s 332ms/step - accuracy: 0.9366 - loss: 0.2115 - val_accuracy: 0.8006 - val_loss: 0.5800
```

### Using the Model

In [26]:
test_ds = tf.keras.utils.image_dataset_from_directory(
    './clothing-dataset-small/test',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(input_size, input_size),
    shuffle=False
)

data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomZoom(0.1),
])

def augment_and_preprocess(image, label):
    image = data_augmentation(image)
    image = preprocess_input(image)
    return image, label

test_ds = test_ds.map(augment_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=tf.data.AUTOTUNE)

Found 372 files belonging to 10 classes.


In [24]:
model = keras.models.load_model('xception_v5_14_0.897.keras')

In [27]:
model.evaluate(test_ds)

12/12 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.8883 - loss: 0.3332


[0.27741047739982605, 0.9032257795333862]

In [28]:

path = 'clothing-dataset-small/test/longsleeve/0f4d9494-612a-48f8-be05-e47f4fc54d54.jpg'


In [29]:
img = load_img(path, target_size=(299, 299))


In [30]:
import numpy as np

x = np.array(img)
X = np.array([x])
X.shape

(1, 299, 299, 3)

In [31]:
X = preprocess_input(X)

In [33]:
pred = model.predict(X)
pred

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step


array([[ 1.3876281 , -4.026394  ,  4.76545   ,  2.6195517 , -2.0764413 ,
         1.6954165 , -5.9548235 , -1.2751677 , -0.16200481,  0.3598975 ]],
      dtype=float32)

In [34]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]


In [37]:
res = dict(zip(classes, pred[0]))
res

{'dress': 1.3876281,
 'hat': -4.026394,
 'longsleeve': 4.76545,
 'outwear': 2.6195517,
 'pants': -2.0764413,
 'shirt': 1.6954165,
 'shoes': -5.9548235,
 'shorts': -1.2751677,
 'skirt': -0.16200481,
 't-shirt': 0.3598975}

In [38]:
max(res.items(), key=lambda i: i[1])

('longsleeve', 4.76545)